# Install Dependencies

In [6]:
!pip install -q unsloth==2025.6.8 vllm==0.8.5.post1 bitsandbytes==0.46.1 trl==0.19.0 sacremoses==0.1.1 tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 7.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 39.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 74.7 MB/s eta 0:00:00ta 0:0

In [7]:
# import unsloth
# import torch
# import trl
# import transformers
# import datasets
# import numpy as np
# import bitsandbytes
# import vllm
# import tensorboard
# import sacremoses

# # Print versions of each library
# print(f"unsloth: {unsloth.__version__}")
# print(f"torch: {torch.__version__}")
# print(f"trl: {trl.__version__}")
# print(f"transformers: {transformers.__version__}")
# print(f"datasets: {datasets.__version__}")
# print(f"numpy: {np.__version__}")
# print(f"bitsandbytes: {bitsandbytes.__version__}")
# print(f"vllm: {vllm.__version__}")
# print(f"tensorboard: {tensorboard.__version__}")
# print(f"sacremoses: {sacremoses.__version__}")
# !python --version

# unsloth: 2025.6.8
# torch: 2.6.0+cu124
# trl: 0.19.0
# transformers: 4.52.4
# datasets: 3.6.0
# numpy: 1.26.4
# bitsandbytes: 0.46.1
# vllm: 0.8.5.post1
# tensorboard: 2.18.0
# sacremoses: 0.1.1
# Python 3.11.13

# Download Dataset

In [ ]:
# To temporary Model hub
from huggingface_hub import HfApi
from huggingface_hub import login
# Initialize API
login("<your_huggingface_token>")
api = HfApi()

In [10]:
import unsloth

from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only
import torch

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

from unsloth import FastModel
from unsloth.chat_templates import get_chat_template

import numpy as np
from datasets import concatenate_datasets, load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-19 01:47:06.295252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752889626.482107      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752889626.534747      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-19 01:47:27 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-19 01:47:27 [__init__.py:239] Automatically detected platform cuda.


In [12]:
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-3-1b-it",
    max_seq_length = max_seq_length,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    fast_inference = True, # Enable vLLM fast inference
    # gpu_memory_utilization = 0.7,
    # device_map = device_map 
)
lora_rank = 16
model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=lora_rank,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

==((====))==  Unsloth 2025.6.8: Fast Gemma3 patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Unsloth: Making `model.base_model.model.model` require gradients


In [14]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

SYSTEM_PROMPT = """
Trả lời theo định dạng sau đây:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""


In [15]:
from datasets import get_dataset_config_names
print(get_dataset_config_names('tmnam20/ViMedAQA'))

dataset = load_dataset('tmnam20/ViMedAQA', 'disease')

README.md: 0.00B [00:00, ?B/s]

['all', 'body-part', 'disease', 'drug', 'medicine']


train-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/501k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/478k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14121 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/784 [00:00<?, ? examples/s]

# Data Processing

In [16]:
# Merge training and validation samples
test_samples = concatenate_datasets([dataset['validation'],
                                         dataset['test']])

# Shuffle the merged dataset with a fixed random seed
training_samples = dataset['train']
test_samples = test_samples.shuffle(seed=42)

print("Number of training samples:", training_samples.num_rows)
print("Number of test samples:", test_samples.num_rows)

Number of training samples: 14121
Number of test samples: 1569


In [17]:
# Define a mapping function to create the prompt and answer fields
def map_fn(x):
    return {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': x['answer'],  # The reference answer
        'question': x['question']
    }

# Apply the mapping function to each split and remove unnecessary columns
training_samples = training_samples.map(map_fn).remove_columns(['question_idx', 'context', 'title', 'keyword', 'topic', 'article_url', 'author', 'author_url'])
test_samples = test_samples.map(map_fn).remove_columns(['question_idx', 'context', 'title', 'keyword', 'topic', 'article_url', 'author', 'author_url'])


Map:   0%|          | 0/14121 [00:00<?, ? examples/s]

Map:   0%|          | 0/1569 [00:00<?, ? examples/s]

In [18]:
training_samples, test_samples

(Dataset({
     features: ['question', 'answer', 'prompt'],
     num_rows: 14121
 }),
 Dataset({
     features: ['question', 'answer', 'prompt'],
     num_rows: 1569
 }))

In [19]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import CrossEncoder
from typing import List
import re

# ------ Device Configuration ------
main_device = "cuda" if torch.cuda.is_available() else "cpu"
reward_device = "cpu" # "cuda" if torch.cuda.is_available() else "cpu"

# ------ Semantic Correctness Reward ------
def semantic_correctness(responses: List[str], answers: List[str]) -> List[float]:
    """Calculate semantic similarity using cross-encoder"""
    model = CrossEncoder('cross-encoder/stsb-roberta-base', device=reward_device)
    with torch.no_grad():
        inputs = list(zip(responses, answers))
        similarities = model.predict(inputs, show_progress_bar=False).tolist()
        # Set similarity to -1 if the response is an empty string
        similarities = [-1.0 if response == "" else similarity for response, similarity in zip(responses, similarities)]
        return similarities

# ------ Efficient Perplexity Calculation ------
class PerplexityCalculator:
    def __init__(self, model_name="microsoft/biogpt", device=reward_device):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.device = device
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.model.eval()

    def calculate(self, texts: List[str], batch_size=8) -> List[float]:
        perplexities = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            try:
                if not batch: continue

                encodings = self.tokenizer(
                    batch,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=200
                ).to(self.device)

                with torch.no_grad():
                    outputs = self.model(**encodings, labels=encodings.input_ids)

                loss = outputs.loss
                if torch.isnan(loss):
                    raise ValueError("NaN loss encountered")

                batch_perplexity = torch.exp(loss).repeat(len(batch)).cpu().tolist()
                # Set perplexity to -1 if the input is an empty string
                batch_perplexity = [-1.0 if text == "" else perplex for text, perplex in zip(batch, batch_perplexity)]
                perplexities.extend(batch_perplexity)

            except Exception as e:
                print(f"Error in batch {i//batch_size}: {str(e)}")
                perplexities.extend([1000.0] * len(batch))

        return perplexities

perplexity_calculator = PerplexityCalculator()

# ------ Tag Presence Reward ------
def tag_presence_reward(completions: List[dict]) -> List[float]:
    """Reward for presence of <reasoning> and <answer> tags"""
    rewards = []
    for completion in completions:
        content = completion[0]['content']
        has_reasoning = bool(re.search(r'<reasoning>.*?</reasoning>', content, re.DOTALL))
        has_answer = bool(re.search(r'<answer>.*?</answer>', content, re.DOTALL))
        reward = 0.5 * has_reasoning + 0.5 * has_answer
        rewards.append(reward)
    return rewards

# ------ Combined Reward Function ------
def combined_reward_func(
    prompts, completions, answer, **kwargs
) -> List[float]:
    # Extract generated responses
    responses = []
    valid_indices = []
    for idx, completion in enumerate(completions):
        try:
            generated_content = completion[0]['content'].strip()

            # Extract only the <answer> content using regex
            answer_match = re.search(r'<answer>(.*?)</answer>', generated_content, re.DOTALL)
            if answer_match:
                generated_content = answer_match.group(1).strip()
            else:
                responses.append("")  # Append empty string to maintain index consistency
                valid_indices.append(idx)
                continue

            # Skip if empty or just repeating the prompt
            user_prompt = prompts[idx][-1]['content']
            if not generated_content or generated_content == user_prompt:
                # print(f"generated_content: continue")
                responses.append("")  # Append empty string to maintain index consistency
                valid_indices.append(idx)
                continue

            responses.append(generated_content)
            # print(f"generated_content3: {generated_content}")
            valid_indices.append(idx)
        except (KeyError, IndexError):
            responses.append("")  # Append empty string to maintain index consistency
            valid_indices.append(idx)
            continue

    if not responses:
        return [-1.0] * len(completions)

    # Calculate rewards
    try:
        # Use raw answers without tag processing
        processed_answers = answer
        # print(f"processed_answers: {processed_answers[0]}")

        similarities = semantic_correctness(responses, [processed_answers[i] for i in valid_indices])
        perplexities = perplexity_calculator.calculate([processed_answers[i] for i in valid_indices])
        tag_rewards = tag_presence_reward([completions[i] for i in valid_indices])
    except Exception as e:
        print(f"Reward calculation error: {str(e)}")
        return [-1.0] * len(completions)

    # Convert to tensors with stability
    sim_scores = torch.nan_to_num(torch.tensor(similarities), nan=0.0)
    perplex_scores = torch.nan_to_num(torch.tensor(perplexities), nan=1000.0)
    tag_scores = torch.tensor(tag_rewards)

    # Perplexity reward calculation
    perplex_rewards = 1 / (perplex_scores / (perplex_scores.mean() + 1e-9))

    # Normalize with stability
    score_range = perplex_rewards.max() - perplex_rewards.min()
    if score_range < 1e-6:
        perplex_rewards_normalized = torch.ones_like(perplex_rewards) * 0.5
    else:
        perplex_rewards_normalized = (perplex_rewards - perplex_rewards.min()) / score_range

    # Combine scores with validation
    combined = [
        0.5 * sim.item() + 0.4 * pr.item() + 0.1 * tag.item()
        for sim, pr, tag in zip(sim_scores, perplex_rewards_normalized, tag_scores)
        if not torch.isnan(sim) and not torch.isnan(pr) and not torch.isnan(tag)
    ]

    # Map back to original indices
    final_rewards = [-1.0] * len(completions)
    for idx, reward in zip(valid_indices, combined):
        final_rewards[idx] = max(min(reward, 1.0), -1.0)  # Clamp between -1 and 1

    assert len(final_rewards) == len(completions), "Reward mapping error"
    return final_rewards

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

In [20]:
tokenized = training_samples.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
# print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
# print("Max Length = ", maximum_length)

# # Filter only samples smaller than 90% max length
# dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Map:   0%|          | 0/14121 [00:00<?, ? examples/s]

Map:   0%|          | 0/14121 [00:00<?, ? examples/s]

# Download Model

In [23]:
from huggingface_hub import HfApi
from huggingface_hub import snapshot_download
import os
api = HfApi()

repo_id = "heboya8/Gemma-3-1B-GRPO-Vi-Medical-LoRA"
save_path = "./Gemma-3-1B-GRPO-Vi-Medical-LoRA"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Download the dataset
snapshot_download(repo_id=repo_id, repo_type="model", local_dir=save_path)

Fetching 840 files:   0%|          | 0/840 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

optimizer.pt:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

(…)ut.tfevents.1752717411.26fb8dd60e61.36.0:   0%|          | 0.00/227k [00:00<?, ?B/s]

(…)ut.tfevents.1752467467.9547df361d69.36.0:   0%|          | 0.00/169k [00:00<?, ?B/s]

(…)ut.tfevents.1752644832.46ca52c001f0.36.0:   0%|          | 0.00/180k [00:00<?, ?B/s]

(…)ut.tfevents.1752839507.f655fdf7a32c.36.0:   0%|          | 0.00/131k [00:00<?, ?B/s]

'/content/Gemma-3-1B-GRPO-Vi-Medical-LoRA'

# Config Training

In [30]:
import os
import wandb
from vllm import SamplingParams
from trl import GRPOConfig, GRPOTrainer
from transformers import TrainingArguments
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Suppress tokenizer parallelism warning

# Define max lengths
# # max_prompt_length = maximum_length + 1 # + 1 just in case!
# # max_prompt_length = maximum_length + 1 # + 1 just in case!
max_prompt_length = 128
max_completion_length = max_seq_length - max_prompt_length

# Custom GRPOTrainer to handle vllm_sampling_params as dict or SamplingParams
class CustomGRPOTrainer(GRPOTrainer):
    def __init__(self, model, processing_class, reward_funcs, args, train_dataset, eval_dataset):
        super().__init__(model=model, processing_class=processing_class, reward_funcs=reward_funcs, 
                        args=args, train_dataset=train_dataset, eval_dataset=eval_dataset)
        # Convert vllm_sampling_params dict to SamplingParams if needed
        if isinstance(args.vllm_sampling_params, dict):
            self.vllm_sampling_params = SamplingParams(**args.vllm_sampling_params)
        else:
            self.vllm_sampling_params = args.vllm_sampling_params


# Convert SamplingParams to a dictionary for JSON serialization
vllm_sampling_params_dict = {
    "min_p": 0.1,
    "top_p": 1.0,
    "top_k": -1,
    "seed": 42,
    "stop": [tokenizer.eos_token],  # Ensure tokenizer is defined
    "include_stop_str_in_output": True,
}

# Custom GRPOConfig to handle vllm_sampling_params serialization
class CustomGRPOConfig(GRPOConfig):  # Inherit from GRPOConfig, not TrainingArguments
    def to_dict(self):
        d = super().to_dict()
        # Ensure vllm_sampling_params is serialized correctly
        if "vllm_sampling_params" in d:
            d["vllm_sampling_params"] = d["vllm_sampling_params"]  # Already a dict
        return d



In [31]:
# Define training arguments
training_args = CustomGRPOConfig(
    resume_from_checkpoint="./Gemma-3-1B-GRPO-Vi-Medical-LoRA",
    vllm_sampling_params=vllm_sampling_params_dict,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    # warmup_ratio = 30,
    warmup_steps=30,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    num_train_epochs = 3, # Set to 1 for a full training run
    # max_steps = 20,
    save_steps = 20,
    report_to = "tensorboard", # Can use Weights & Biases
    output_dir = "./Gemma-3-1B-GRPO-Vi-Medical-LoRA",
    fp16=True,  
    bf16=False,
    # For optional training + evaluation
    # fp16_full_eval = True,
    per_device_eval_batch_size = 8,
    # eval_accumulation_steps = 2,
    eval_strategy = "steps",
    eval_steps = 200,
)


In [33]:
# Initialize trainer
trainer = CustomGRPOTrainer(
    model=model,  # Ensure model is defined
    processing_class=tokenizer,  # Ensure tokenizer is defined
    reward_funcs=[combined_reward_func],  # Ensure combined_reward_func is defined
    args=training_args,
    train_dataset=training_samples,  # Ensure training_samples is defined
    eval_dataset=test_samples,  # Ensure test_samples is defined
)


Unsloth: Switching to float32 training since model cannot work with float16


# Start Training

In [34]:
# Train the model
trainer.train(resume_from_checkpoint=True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,121 | Num Epochs = 3 | Total steps = 1,326
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 13,045,760 of 1,000,000,000 (1.30% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 32768, 'top_p': 0.95}. If this is not desired, please set these values explicitly.
You have set `compile_config`, but we are unable to meet the criteria for compilation. Compilation will be skipped.


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Error in batch 7: Expected a cuda device, but got: cpu
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / combined_reward_func / mean,rewards / combined_reward_func / std
1200,0.000200,0.000166,0,0,0,0,0,0,0,0,0,0,0,0


Error in batch 0: Expected a cuda device, but got: cpu
Error in batch 1: Expected a cuda device, but got: cpu
Error in batch 2: Expected a cuda device, but got: cpu
Error in batch 3: Expected a cuda device, but got: cpu
Error in batch 4: Expected a cuda device, but got: cpu
Error in batch 5: Expected a cuda device, but got: cpu
Error in batch 6: Expected a cuda device, but got: cpu
Error in batch 7: Expected a cuda device, but got: cpu
Error in batch 0: Expected a cuda device, but got: cpu
Error in batch 1: Expected a cuda device, but got: cpu
Error in batch 2: Expected a cuda device, but got: cpu
Error in batch 3: Expected a cuda device, but got: cpu
Error in batch 4: Expected a cuda device, but got: cpu
Error in batch 5: Expected a cuda device, but got: cpu
Error in batch 6: Expected a cuda device, but got: cpu
Error in batch 7: Expected a cuda device, but got: cpu
Error in batch 0: Expected a cuda device, but got: cpu
Error in batch 1: Expected a cuda device, but got: cpu
Error in b

TrainOutput(global_step=1325, training_loss=1.8164746766417937e-05, metrics={'train_runtime': 33862.4286, 'train_samples_per_second': 1.251, 'train_steps_per_second': 0.039, 'total_flos': 0.0, 'train_loss': 1.8164746766417937e-05})

In [ ]:
model.save_pretrained("Gemma-3-1B-GRPO-Vi-Medical-LoRA")  # Local saving
tokenizer.save_pretrained("Gemma-3-1B-GRPO-Vi-Medical-LoRA")

('Gemma-3-1B-GRPO-Vi-Medical-LoRA/tokenizer_config.json',
 'Gemma-3-1B-GRPO-Vi-Medical-LoRA/special_tokens_map.json',
 'Gemma-3-1B-GRPO-Vi-Medical-LoRA/chat_template.jinja',
 'Gemma-3-1B-GRPO-Vi-Medical-LoRA/tokenizer.model',
 'Gemma-3-1B-GRPO-Vi-Medical-LoRA/added_tokens.json',
 'Gemma-3-1B-GRPO-Vi-Medical-LoRA/tokenizer.json')

# Inference

In [ ]:
question = ("Khi nghi ngờ bị loét dạ dày tá tràng nên đến khoa nào "
            "tại bệnh viện để thăm khám?")
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user",   "content": question},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,  # Must add for generation
    tokenize=False,
)

from transformers import TextStreamer

_ = model.generate(
    **tokenizer(text, return_tensors="pt").to(device),
    max_new_tokens=2048,  # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature=1.0, top_p=0.95, top_k=64,
    streamer=TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True),
)